In [2]:
pip install PyPDF2 python-docx pandas


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 12.1 MB/s eta 0:00:00


In [3]:
pip install pdf2image opencv-python pillow


In [4]:
!apt-get update
!apt-get install libssl-dev
!apt-get install poppler-utils


Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:9 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [3,323 kB]
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,172 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:13 http://security.ubuntu.com/ub

In [5]:
!pip install python-docx
!pip install pdf2image
!pip install fpdf
!apt-get install poppler-utils
!pip install python-docx pdf2image fpdf
!pip install aspose-words



  Preparing metadata (setup.py) ... done
  Created wheel for fpdf: filename=fpdf-1.7.2-py2.py3-none-any.whl size=40704 sha256=a3ab78d1148f44f01c6b8f20b5db0bfaac5bf2f3d696482ba6eeffa4da615876
  Stored in directory: /root/.cache/pip/wheels/f9/95/ba/f418094659025eb9611f17cbcaf2334236bf39a0c3453ea455
Successfully built fpdf
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
poppler-utils is already the newest version (22.02.0-2ubuntu0.5).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.2/84.2 MB 9.1 MB/s eta 0:00:00


In [8]:
import os
import re
import pandas as pd
from PyPDF2 import PdfReader
from docx import Document

# Montar Google Drive
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [19]:
import os
from pdf2image import convert_from_path
import pandas as pd

# Directorios
dataset_folder = "/content/drive/MyDrive/curriculum_viate_dataset/pdf"  # Cambia 'word' por 'pdf'
output_folder = "/content/drive/MyDrive/curriculum_viate_dataset/output_file_dataset"
csv_output_path = "/content/drive/MyDrive/curriculum_viate_dataset/dataset_images.csv"

# Crear la carpeta de salida si no existe
os.makedirs(output_folder, exist_ok=True)

# Función para verificar y limpiar nombres de archivos
def clean_filename(filename):
    """
    Limpia el nombre del archivo eliminando caracteres problemáticos.
    """
    return "".join(c for c in filename if c.isalnum() or c in (' ', '.', '_')).rstrip()

# Función para convertir PDF a imágenes
def pdf_to_images(pdf_path, output_prefix, dpi=150):
    """
    Convierte un archivo PDF a imágenes usando pdf2image.
    """
    try:
        # Convertir PDF a imágenes
        pages = convert_from_path(pdf_path, dpi=dpi)
        images = []
        for i, page in enumerate(pages):
            output_image_path = os.path.join(output_folder, f"{output_prefix}_page_{i + 1}.jpg")
            page.save(output_image_path, "JPEG")
            images.append(output_image_path)
        return images
    except Exception as e:
        print(f"Error al convertir {pdf_path} a imágenes: {e}")
        return []

# Procesar el dataset en batches
batch_size = 50  # Tamaño del batch para minimizar el uso de memoria
all_files = [f for f in os.listdir(dataset_folder) if f.endswith(".pdf")]
total_batches = len(all_files) // batch_size + (1 if len(all_files) % batch_size != 0 else 0)

# Verificar si existe un CSV previo para reiniciar desde ahí
if os.path.exists(csv_output_path):
    df_existing = pd.read_csv(csv_output_path)
    processed_files = set(df_existing["filename"].str.split("_page_").str[0])  # Archivos ya procesados
else:
    df_existing = pd.DataFrame(columns=["filename"])
    processed_files = set()

# Procesar archivos en batches
for batch_num in range(total_batches):
    start_idx = batch_num * batch_size
    end_idx = start_idx + batch_size
    batch_files = all_files[start_idx:end_idx]

    filenames = []

    print(f"Procesando batch {batch_num + 1}/{total_batches}...")

    for file in batch_files:
        # Limpia el nombre del archivo
        file = clean_filename(file)

        if file in processed_files:
            print(f"Archivo {file} ya procesado, saltando...")
            continue

        pdf_path = os.path.join(dataset_folder, file)
        output_prefix = os.path.splitext(file)[0]  # Usar el nombre del archivo como prefijo

        try:
            # Convierte el archivo PDF a imágenes
            image_paths = pdf_to_images(pdf_path, output_prefix)
            filenames.extend(image_paths)
        except Exception as e:
            print(f"Error procesando {file}: {e}")

    # Guardar los resultados del batch en el CSV
    batch_data = {"filename": filenames}
    df_batch = pd.DataFrame(batch_data)
    df_existing = pd.concat([df_existing, df_batch], ignore_index=True)
    df_existing.to_csv(csv_output_path, index=False)

    print(f"Batch {batch_num + 1}/{total_batches} procesado y guardado.")

print("Todos los batches procesados. Dataset completo generado.")



Procesando batch 1/66...
Batch 1/66 procesado y guardado.
Procesando batch 2/66...
Batch 2/66 procesado y guardado.
Procesando batch 3/66...
Batch 3/66 procesado y guardado.
Procesando batch 4/66...
Batch 4/66 procesado y guardado.
Procesando batch 5/66...
Batch 5/66 procesado y guardado.
Procesando batch 6/66...
Batch 6/66 procesado y guardado.
Procesando batch 7/66...
Batch 7/66 procesado y guardado.
Procesando batch 8/66...
Batch 8/66 procesado y guardado.
Procesando batch 9/66...
Batch 9/66 procesado y guardado.
Procesando batch 10/66...
Batch 10/66 procesado y guardado.
Procesando batch 11/66...
Batch 11/66 procesado y guardado.
Procesando batch 12/66...
Batch 12/66 procesado y guardado.
Procesando batch 13/66...
Batch 13/66 procesado y guardado.
Procesando batch 14/66...
Batch 14/66 procesado y guardado.
Procesando batch 15/66...
Batch 15/66 procesado y guardado.
Procesando batch 16/66...
Batch 16/66 procesado y guardado.
Procesando batch 17/66...
Batch 17/66 procesado y guardado

In [3]:
import os
from pdf2image import convert_from_path
import pandas as pd

# Directorios de entrada y salida
similar_folder = "/content/drive/MyDrive/curriculum_viate_dataset/curriculum_recomendados_pdf"  # Carpeta de currículums similares
not_similar_folder = "/content/drive/MyDrive/curriculum_viate_dataset/curriculum_no_recomendados_pdf"  # Carpeta de currículums no similares
output_folder = "/content/drive/MyDrive/curriculum_viate_dataset/output_file_supervisado"  # Carpeta donde se guardarán las imágenes generadas

# Crear carpetas de salida si no existen
os.makedirs(output_folder, exist_ok=True)

# Función para convertir PDF a imágenes
def pdf_to_images(pdf_path, output_prefix, dpi=600):
    """
    Convierte un archivo PDF a imágenes usando pdf2image.
    """
    try:
        # Convertir PDF a imágenes
        pages = convert_from_path(pdf_path, dpi=dpi)
        images = []
        for i, page in enumerate(pages):
            output_image_path = os.path.join(output_folder, f"{output_prefix}_page_{i + 1}.jpg")
            page.save(output_image_path, "JPEG")
            images.append(output_image_path)
        return images
    except Exception as e:
        print(f"Error al convertir {pdf_path} a imágenes: {e}")
        return []

# Listas para almacenar nombres de imágenes y etiquetas
filenames = []
labels = []

# Procesar documentos similares
print("Procesando currículums similares...")
for file in os.listdir(similar_folder):
    if file.endswith(".pdf"):
        pdf_path = os.path.join(similar_folder, file)
        output_prefix = f"{os.path.splitext(file)[0]}_similar"
        image_paths = pdf_to_images(pdf_path, output_prefix)
        filenames.extend(image_paths)
        labels.extend([1] * len(image_paths))  # Etiqueta 1 para similares

# Procesar documentos no similares
print("Procesando currículums no similares...")
for file in os.listdir(not_similar_folder):
    if file.endswith(".pdf"):
        pdf_path = os.path.join(not_similar_folder, file)
        output_prefix = f"{os.path.splitext(file)[0]}_no_similar"
        image_paths = pdf_to_images(pdf_path, output_prefix)
        filenames.extend(image_paths)
        labels.extend([0] * len(image_paths))  # Etiqueta 0 para no similares

# Crear un DataFrame con las imágenes generadas y sus etiquetas
data = {"filename": filenames, "label": labels}
df = pd.DataFrame(data)

# Guardar el DataFrame como archivo CSV
output_csv = "/content/drive/MyDrive/curriculum_viate_dataset/labels.csv"
df.to_csv(output_csv, index=False)

print(f"Imágenes generadas y etiquetas guardadas en {output_csv}")



Procesando currículums similares...
Procesando currículums no similares...
Imágenes generadas y etiquetas guardadas en /content/drive/MyDrive/curriculum_viate_dataset/labels.csv


se guardan los datos etiquetados en labels.csv

In [5]:
import numpy as np
import os
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.resnet import preprocess_input
from tensorflow.keras.layers import GlobalAveragePooling2D
import pandas as pd

# Configurar el extractor con pooling
base_model = ResNet50(weights="imagenet", include_top=False)
feature_extractor = Model(inputs=base_model.input, outputs=GlobalAveragePooling2D()(base_model.output))
# Función para cargar un batch de imágenes con mayor detalle
def load_images_in_batch(image_paths, target_size=(299, 299)):
    batch_images = []
    for img_path in image_paths:
        try:
            img = load_img(img_path, target_size=target_size)
            img_array = img_to_array(img)
            batch_images.append(img_array)
        except Exception as e:
            print(f"Error al cargar la imagen {img_path}: {e}")
            batch_images.append(np.zeros((*target_size, 3)))  # Imagen vacía si hay error
    batch_images = np.array(batch_images)
    return preprocess_input(batch_images)

# Directorios y CSV
csv_path = "/content/drive/MyDrive/curriculum_viate_dataset/labels.csv"
output_csv_features = "/content/drive/MyDrive/curriculum_viate_dataset/dataset_features_flattened.csv"

# Cargar el archivo CSV con las etiquetas
data = pd.read_csv(csv_path)

# Inicializar listas para características, etiquetas y nombres de archivos
features = []
labels = []
filenames = []

# Configurar batch size
batch_size = 16
total_batches = len(data) // batch_size + (1 if len(data) % batch_size != 0 else 0)

# Procesar las imágenes en batches
for batch_num in range(total_batches):
    start_idx = batch_num * batch_size
    end_idx = min((batch_num + 1) * batch_size, len(data))
    batch_data = data.iloc[start_idx:end_idx]

    image_paths = batch_data["filename"].tolist()
    batch_labels = batch_data["label"].tolist()

    print(f"Procesando batch {batch_num + 1}/{total_batches}...")

    # Cargar y preprocesar imágenes en batch
    batch_images = load_images_in_batch(image_paths)

    # Extraer características para el batch
    batch_features = feature_extractor.predict(batch_images)

    # Aplanar características
    flattened_features = batch_features.reshape(batch_features.shape[0], -1)  # Convertir a 2D

    # Guardar las características, etiquetas y nombres de archivos
    features.extend(flattened_features)
    labels.extend(batch_labels)
    filenames.extend(image_paths)

# Crear un DataFrame para guardar las características y etiquetas
features_df = pd.DataFrame(features)  # Ahora es 2D
features_df["label"] = labels
features_df["filename"] = filenames

# Guardar como archivo CSV
features_df.to_csv(output_csv_features, index=False)

print(f"Características extraídas y guardadas en {output_csv_features}")




Procesando batch 1/7...
1/1 ━━━━━━━━━━━━━━━━━━━━ 9s 9s/step
Procesando batch 2/7...
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
Procesando batch 3/7...
1/1 ━━━━━━━━━━━━━━━━━━━━ 7s 7s/step
Procesando batch 4/7...
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
Procesando batch 5/7...
1/1 ━━━━━━━━━━━━━━━━━━━━ 7s 7s/step
Procesando batch 6/7...
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
Procesando batch 7/7...
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
Características extraídas y guardadas en /content/drive/MyDrive/curriculum_viate_dataset/dataset_features_flattened.csv


In [6]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.layers import Input
from tensorflow.keras.optimizers import SGD
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=True)

# Ruta al archivo CSV con características
csv_features_path = "/content/drive/MyDrive/curriculum_viate_dataset/dataset_features_flattened.csv"

# Cargar el archivo CSV
data = pd.read_csv(csv_features_path)

# Separar características (features) y etiquetas (labels)
features = data.iloc[:, :-2].values  # Todas las columnas excepto 'label' y 'filename'
labels = data["label"].values        # La columna de etiquetas (1 o 0)

# Dividir en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

# Crear un nuevo modelo para características de tamaño (2048,)
model = Sequential([
    Input(shape=(2048,)),  # Cambiado al tamaño correcto de las características
    Dense(512, activation="relu"),
    Dropout(0.1),
    Dense(256, activation="relu"),
    Dropout(0.1),
    Dense(128, activation="relu"),
    Dropout(0.1),
    Dense(1, activation="sigmoid")
])

# Create an instance of the SGD optimizer with the desired learning rate
optimizer = SGD(learning_rate=0.001)

# Compilar el modelo
model.compile(optimizer=optimizer, loss="binary_crossentropy", metrics=["accuracy","AUC"])

# Entrenar el modelo
model.fit(X_train, y_train, epochs=100, batch_size=64, validation_data=(X_test, y_test), callbacks=[early_stopping])

# Evaluar el modelo
y_pred = (model.predict(X_test) > 0.5).astype("int32")
accuracy = accuracy_score(y_test, y_pred)
print(f"Precisión del modelo: {accuracy * 100:.2f}%")



Epoch 1/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 382ms/step - AUC: 0.4176 - accuracy: 0.4792 - loss: 0.7543 - val_AUC: 0.5144 - val_accuracy: 0.2381 - val_loss: 0.7462
Epoch 2/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step - AUC: 0.4610 - accuracy: 0.4867 - loss: 0.7332 - val_AUC: 0.5240 - val_accuracy: 0.2857 - val_loss: 0.7404
Epoch 3/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - AUC: 0.4398 - accuracy: 0.5133 - loss: 0.7194 - val_AUC: 0.5240 - val_accuracy: 0.3333 - val_loss: 0.7319
Epoch 4/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step - AUC: 0.5476 - accuracy: 0.5504 - loss: 0.6986 - val_AUC: 0.5240 - val_accuracy: 0.3810 - val_loss: 0.7239
Epoch 5/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - AUC: 0.5469 - accuracy: 0.4837 - loss: 0.6961 - val_AUC: 0.5288 - val_accuracy: 0.3810 - val_loss: 0.7218
Epoch 6/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - AUC: 0.5706 - accuracy: 0.5719 - loss: 0.6887 - val_AUC: 0.5337 - val_accuracy: 0.3810 - val_loss: 0.7129
Epoch 7/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step -

In [7]:
import tensorflow as tf
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import load_model

# Ruta para guardar el modelo
model_save_path = "/content/drive/MyDrive/curriculum_viate_dataset/trained_model.h5"

# Guardar el modelo entrenado
model.save(model_save_path)
print(f"Modelo guardado en: {model_save_path}")

# Para cargar el modelo guardado más adelante
loaded_model = load_model(model_save_path)

# Verificar el modelo cargado evaluándolo con los datos de prueba
y_pred_loaded = (loaded_model.predict(X_test) > 0.5).astype("int32")
accuracy_loaded = accuracy_score(y_test, y_pred_loaded)
print(f"Precisión del modelo cargado: {accuracy_loaded * 100:.2f}%")


Modelo guardado en: /content/drive/MyDrive/curriculum_viate_dataset/trained_model.h5


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
Precisión del modelo cargado: 61.90%


In [9]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.resnet import preprocess_input
from tensorflow.keras.models import Model

# Ruta del modelo entrenado y datos no etiquetados
model_path = "/content/drive/MyDrive/curriculum_viate_dataset/trained_model.h5"  # Cambia a la ruta de tu modelo
unlabeled_csv_path = "/content/drive/MyDrive/curriculum_viate_dataset/dataset_images.csv"  # Dataset sin etiquetas
output_csv_path = "/content/drive/MyDrive/curriculum_viate_dataset/dataset_labels_with_predictions.csv"

# Cargar el modelo entrenado
model = load_model(model_path)

# Cargar el extractor de características ResNet50
base_model = ResNet50(weights="imagenet", include_top=False, pooling="avg")
feature_extractor = Model(inputs=base_model.input, outputs=base_model.output)

# Función para cargar y preprocesar imágenes en batch
def preprocess_and_extract_features_batch(image_paths, target_size=(299, 299)):
    """
    Preprocesa un batch de imágenes y extrae sus características.
    """
    batch_images = []
    for image_path in image_paths:
        try:
            img = load_img(image_path, target_size=target_size)
            img_array = img_to_array(img)
            batch_images.append(img_array)
        except Exception as e:
            print(f"Error al cargar la imagen {image_path}: {e}")
            batch_images.append(np.zeros((*target_size, 3)))  # Imagen vacía si hay error
    batch_images = np.array(batch_images)
    batch_images = preprocess_input(batch_images)
    features = feature_extractor.predict(batch_images)  # Extraer características en batch
    return features

# Cargar datos sin etiquetas
data = pd.read_csv(unlabeled_csv_path)

# Configurar batch size
batch_size = 32
total_batches = len(data) // batch_size + (1 if len(data) % batch_size != 0 else 0)

# Inicializar lista para almacenar etiquetas predichas
predicted_labels = []

# Procesar y predecir por batches
for batch_num in range(total_batches):
    start_idx = batch_num * batch_size
    end_idx = min((batch_num + 1) * batch_size, len(data))
    batch_data = data.iloc[start_idx:end_idx]

    image_paths = batch_data["filename"].tolist()

    print(f"Procesando batch {batch_num + 1}/{total_batches}...")

    # Preprocesar imágenes y extraer características
    try:
        batch_features = preprocess_and_extract_features_batch(image_paths)
        # Predecir etiquetas en batch
        predictions = model.predict(batch_features)
        batch_labels = [1 if pred[0] > 0.5 else 0 for pred in predictions]  # Etiquetas binarias
        predicted_labels.extend(batch_labels)
    except Exception as e:
        print(f"Error al procesar el batch {batch_num + 1}: {e}")
        predicted_labels.extend([None] * len(image_paths))  # Etiqueta como None si hay un error

# Agregar etiquetas predichas al DataFrame
data["label"] = predicted_labels

# Guardar el nuevo archivo CSV
data.to_csv(output_csv_path, index=False)

print(f"Etiquetas generadas y guardadas en {output_csv_path}")


94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
Procesando batch 1/919...
1/1 ━━━━━━━━━━━━━━━━━━━━ 14s 14s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 119ms/step
Procesando batch 2/919...
1/1 ━━━━━━━━━━━━━━━━━━━━ 11s 11s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
Procesando batch 3/919...
1/1 ━━━━━━━━━━━━━━━━━━━━ 10s 10s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
Procesando batch 4/919...
1/1 ━━━━━━━━━━━━━━━━━━━━ 10s 10s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
Procesando batch 5/919...
1/1 ━━━━━━━━━━━━━━━━━━━━ 10s 10s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
Procesando batch 6/919...
1/1 ━━━━━━━━━━━━━━━━━━━━ 9s 9s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
Procesando batch 7/919...
1/1 ━━━━━━━━━━━━━━━━━━━━ 11s 11s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
Procesando batch 8/919...
1/1 ━━━━━━━━━━━━━━━━━━━━ 11s 11s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
Procesando batch 9/919...
1/1 ━━━━━━━━━━━━━━━━━━━━ 11s 11s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
Procesando batch 10/919...
1/1 ━

In [ ]:
import numpy as np
import os
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.efficientnet import preprocess_input
import pandas as pd

# Cargar el modelo preentrenado EfficientNetB0
base_model = EfficientNetB0(weights="imagenet", include_top=False, pooling="avg")
feature_extractor = Model(inputs=base_model.input, outputs=base_model.output)

# Función para cargar un batch de imágenes
def load_images_in_batch(image_paths, target_size=(224, 224)):
    """
    Carga un batch de imágenes y las preprocesa para el modelo EfficientNetB0.
    """
    batch_images = []
    for img_path in image_paths:
        try:
            img = load_img(img_path, target_size=target_size)
            img_array = img_to_array(img)
            batch_images.append(img_array)
        except Exception as e:
            print(f"Error al cargar la imagen {img_path}: {e}")
            batch_images.append(np.zeros((*target_size, 3)))  # Imagen vacía si hay error
    batch_images = np.array(batch_images)
    return preprocess_input(batch_images)

# Directorios y CSV
csv_path = "/content/drive/MyDrive/curriculum_viate_dataset/labels.csv"
output_csv_features = "/content/drive/MyDrive/curriculum_viate_dataset/dataset_features.csv"

# Cargar el archivo CSV con las etiquetas
data = pd.read_csv(csv_path)

# Inicializar listas para características, etiquetas y nombres de archivos
features = []
labels = []
filenames = []

# Configurar batch size
batch_size = 32
total_batches = len(data) // batch_size + (1 if len(data) % batch_size != 0 else 0)

# Procesar las imágenes en batches
for batch_num in range(total_batches):
    start_idx = batch_num * batch_size
    end_idx = min((batch_num + 1) * batch_size, len(data))
    batch_data = data.iloc[start_idx:end_idx]

    image_paths = batch_data["filename"].tolist()
    batch_labels = batch_data["label"].tolist()

    print(f"Procesando batch {batch_num + 1}/{total_batches}...")

    # Cargar y preprocesar imágenes en batch
    batch_images = load_images_in_batch(image_paths)

    # Extraer características para el batch
    batch_features = feature_extractor.predict(batch_images)

    # Guardar las características, etiquetas y nombres de archivos
    features.extend(batch_features)
    labels.extend(batch_labels)
    filenames.extend(image_paths)

# Crear un DataFrame para guardar las características y etiquetas
features_df = pd.DataFrame(features)
features_df["label"] = labels
features_df["filename"] = filenames

# Guardar como archivo CSV
features_df.to_csv(output_csv_features, index=False)

print(f"Características extraídas y guardadas en {output_csv_features}")


In [ ]:
import os
import pandas as pd

# Directorios de las carpetas
similar_folder = "/content/drive/MyDrive/curriculum_viate_dataset/curriculum_recomendados"  # Cambia esto a la carpeta de currículums similares
not_similar_folder = "/content/drive/MyDrive/curriculum_viate_dataset/CV_norecomendados"  # Cambia esto a la carpeta de currículums no similares

# Crear listas para almacenar nombres de archivos y etiquetas
filenames = []
labels = []

# Etiquetar archivos de la carpeta "similares"
for file in os.listdir(similar_folder):
    if file.endswith((".jpg", ".jpeg", ".png")):  # Filtrar imágenes
        filenames.append(os.path.join(similar_folder, file))
        labels.append(1)  # Etiqueta: 1 para "similar"

# Etiquetar archivos de la carpeta "no similares"
for file in os.listdir(not_similar_folder):
    if file.endswith((".jpg", ".jpeg", ".png")):  # Filtrar imágenes
        filenames.append(os.path.join(not_similar_folder, file))
        labels.append(0)  # Etiqueta: 0 para "no similar"

# Crear un DataFrame con los nombres de los archivos y etiquetas
data = {"filename": filenames, "label": labels}
df = pd.DataFrame(data)

# Guardar el DataFrame como archivo CSV
output_csv = "/path/to/labels.csv"  # Ruta donde quieres guardar el archivo CSV
df.to_csv(output_csv, index=False)

print(f"Archivo CSV generado con {len(filenames)} registros. Guardado en: {output_csv}")


In [ ]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import preprocess_input

# Directorio base del dataset
base_dir = "/content/drive/MyDrive/curriculum_viate_dataset/Output_file"

# Tamaño de imagen y batch
IMG_SIZE = (224, 224)
BATCH_SIZE = 32

# Generador de datos con aumento (opcional)
datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,  # Normaliza las imágenes para ResNet
    validation_split=0.2                     # Divide los datos en entrenamiento y validación
)

# Cargar datos de entrenamiento y validación
train_data = datagen.flow_from_directory(
    base_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training'  # Subconjunto de entrenamiento
)

val_data = datagen.flow_from_directory(
    base_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation'  # Subconjunto de validación
)


Found 0 images belonging to 0 classes.
Found 0 images belonging to 0 classes.


In [ ]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten

# Modelo base de ResNet50 sin las capas superiores
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Construir modelo
model = Sequential([
    base_model,                # Modelo base (extractor de características)
    Flatten(),                 # Aplanar las características
    Dense(256, activation='relu'),  # Capa densa intermedia
    Dense(3, activation='softmax')  # 3 clases (Simple, Intermedio, Complejo)
])

# Congelar capas de ResNet (opcional, para usar solo características preentrenadas)
base_model.trainable = False

# Compilar el modelo
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Resumen del modelo
model.summary()


94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ resnet50 (Functional)                │ (None, 7, 7, 2048)          │      23,587,712 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 100352)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 256)                 │      25,690,368 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 3)                   │             771 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 49,278,851 (187.98 MB)

 Trainable params: 25,691,139 (98.00 MB)

 Non-trainable params: 23,587,712 (89.98 MB)

In [ ]:
# Entrenamiento
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=10,  # Ajusta el número de épocas según el dataset
    steps_per_epoch=train_data.samples // BATCH_SIZE,
    validation_steps=val_data.samples // BATCH_SIZE
)


ValueError: Must provide at least one structure

In [ ]:
# Evaluar con el conjunto de validación
loss, accuracy = model.evaluate(val_data)
print(f"Precisión en validación: {accuracy:.2f}")


ValueError: Must provide at least one structure

In [ ]:
import os
from pdf2image import convert_from_path
import cv2
import numpy as np

# Configuración de directorios
PDF_FOLDER = "/content/drive/MyDrive/curriculum_viate_dataset/word"  # Carpeta donde están los PDFs
OUTPUT_FOLDER = "/content/drive/MyDrive/curriculum_viate_dataset/Output_file"  # Carpeta para guardar las imágenes procesadas
IMAGE_SIZE = (224, 224)  # Tamaño al que se redimensionarán las imágenes

# Función para convertir PDFs a imágenes
def pdf_to_images(pdf_folder, output_folder):
    """
    Convierte todos los PDFs de una carpeta en imágenes y las guarda en el directorio de salida.
    """
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for pdf_file in os.listdir(pdf_folder):
        if pdf_file.endswith(".pdf"):
            pdf_path = os.path.join(pdf_folder, pdf_file)
            images = convert_from_path(pdf_path)
            for i, image in enumerate(images):
                output_path = os.path.join(output_folder, f"{os.path.splitext(pdf_file)[0]}_page_{i + 1}.jpg")
                image.save(output_path, "JPEG")
                print(f"Guardado: {output_path}")

# Función para preprocesar imágenes
def preprocess_image(image_path, image_size):
    """
    Preprocesa una imagen: redimensiona y normaliza los píxeles.
    """
    image = cv2.imread(image_path)  # Leer la imagen
    image = cv2.resize(image, image_size)  # Redimensionar
    image = image / 255.0  # Normalizar valores entre 0 y 1
    return image

# Función para preprocesar un conjunto de imágenes
def preprocess_images(image_folder, image_size):
    """
    Preprocesa todas las imágenes en una carpeta y devuelve un array numpy.
    """
    processed_images = []
    for img_file in os.listdir(image_folder):
        if img_file.endswith((".jpg", ".jpeg", ".png")):
            img_path = os.path.join(image_folder, img_file)
            processed_image = preprocess_image(img_path, image_size)
            processed_images.append(processed_image)
    return np.array(processed_images)

# Ejecución del flujo
if __name__ == "__main__":
    # 1. Convertir PDFs a imágenes
    pdf_to_images(PDF_FOLDER, OUTPUT_FOLDER)

    # 2. Preprocesar las imágenes generadas
    processed_images = preprocess_images(OUTPUT_FOLDER, IMAGE_SIZE)
    print(f"Total de imágenes preprocesadas: {len(processed_images)}")

    # Guardar el dataset preprocesado si es necesario
    np.save("processed_images.npy", processed_images)
    print("Imágenes preprocesadas guardadas en 'processed_images.npy'.")

Total de imágenes preprocesadas: 0
Imágenes preprocesadas guardadas en 'processed_images.npy'.


In [ ]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten

num_classes = 3

base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
model = Sequential([
    base_model,
    Flatten(),
    Dense(256, activation='relu'),
    Dense(num_classes, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
import numpy as np
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.models import Model
import cv2
from sklearn.metrics.pairwise import cosine_similarity

# 1. Modelo preentrenado para extracción de características
base_model = ResNet50(weights='imagenet', include_top=False, pooling='avg')
feature_extractor = Model(inputs=base_model.input, outputs=base_model.output)

# 2. Función para extraer características
def extract_features(image_path, model):
    """
    Extrae características de una imagen usando un modelo preentrenado.
    """
    image = cv2.imread(image_path)
    image = cv2.resize(image, (224, 224))
    image = preprocess_input(image)
    image = np.expand_dims(image, axis=0)
    features = model.predict(image)
    return features

# 3. Seleccionar un currículum de referencia
reference_image_path = "path/to/reference_cv.jpg"  # Imagen de currículum de referencia
reference_features = extract_features(reference_image_path, feature_extractor)

# 4. Clasificar otros currículums según su similitud con el de referencia
dataset_folder = "path/to/dataset"
similarity_threshold = 0.8  # Ajusta según tu criterio

similar_cvs = []

for img_file in os.listdir(dataset_folder):
    if img_file.endswith((".jpg", ".jpeg", ".png")):
        img_path = os.path.join(dataset_folder, img_file)
        features = extract_features(img_path, feature_extractor)
        similarity = cosine_similarity(reference_features, features)[0][0]
        if similarity >= similarity_threshold:
            similar_cvs.append(img_path)

print(f"Currículums similares encontrados: {len(similar_cvs)}")
print("Listado de currículums similares:", similar_cvs)


In [ ]:
import os
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import TensorBoard
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
# 1. Cargar el dataset preprocesado
# Asegúrate de haber generado y guardado el archivo "processed_images.npy"
processed_images = np.load("processed_images.npy")  # Imágenes preprocesadas
labels = np.load("labels.npy")  # Etiquetas correspondientes

# Número de clases (categorías)
num_classes = len(np.unique(labels))

# Convertir las etiquetas en formato categórico
labels_categorical = to_categorical(labels, num_classes=num_classes)

# 2. Dividir los datos en conjuntos de entrenamiento, validación y prueba
X_train, X_temp, y_train, y_temp = train_test_split(processed_images, labels_categorical, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

print(f"Tamaño del conjunto de entrenamiento: {X_train.shape[0]}")
print(f"Tamaño del conjunto de validación: {X_val.shape[0]}")
print(f"Tamaño del conjunto de prueba: {X_test.shape[0]}")

# 4. Configurar TensorBoard
tensorboard = TensorBoard(log_dir="logs", histogram_freq=1)

# 5. Entrenamiento del modelo
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=10,  # Cambia a más épocas según sea necesario
    batch_size=32,
    callbacks=[tensorboard]
)

# 6. Evaluar el modelo
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Precisión en el conjunto de prueba: {test_accuracy:.2f}")

# 7. Métricas adicionales: matriz de confusión y clasificación
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_test, axis=1)

print("\nReporte de clasificación:")
print(classification_report(y_true_classes, y_pred_classes))

# Matriz de confusión
conf_matrix = confusion_matrix(y_true_classes, y_pred_classes)
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, cmap="Blues", fmt="d", xticklabels=range(num_classes), yticklabels=range(num_classes))
plt.title("Matriz de confusión")
plt.xlabel("Predicción")
plt.ylabel("Real")
plt.show()



NameError: name 'X_train' is not defined